In [1]:
#libraries
#%matplotlib notebook

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from IPython.core.display import clear_output
from tensorflow.examples.tutorials.mnist import input_data

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
sess = tf.InteractiveSession()

In [78]:
mnist = input_data.read_data_sets("MNIST_data/")
train_x = np.concatenate((mnist.train.images[mnist.train.labels == 0],mnist.train.images[mnist.train.labels == 1],mnist.train.images[mnist.train.labels == 2],mnist.train.images[mnist.train.labels == 6]))
# train_x = np.array(train_x); mu = np.mean(train_x); sigma = np.std(train_x); #data standardization
# train_x = (train_x - mu + 0.0)/sigma

n_train = train_x.shape[0]
n0 = np.sum(mnist.train.labels == 0); n1 = np.sum(mnist.train.labels == 1); n2 = np.sum(mnist.train.labels == 2); n3 = np.sum(mnist.train.labels == 5)
e1 = n0+n1; e2 = n0+n1+n2;
input_size = train_x.shape[1]
x = tf.placeholder(tf.float32, [None, input_size])
momentum = tf.placeholder(tf.float32,[])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [79]:
hidden_shape = [150,2,150] #No of neurons in each hidden layer
act_fns = ['tanh','tanh','tanh'] #Activation functions of layers
code_lyr = 2 #index of hidden layer containing sparse vector
num_lyr = len(hidden_shape) #No of hidden layers
net_shape = [input_size] + hidden_shape + [input_size]
learning_rate = tf.placeholder(tf.float32,[])
beta = tf.placeholder(tf.float32,[]) #sparsity_penalty weight

In [80]:
#parameters
params = {'W':{},'b':{}} #W: Weights, b: biases
for lay_num in range(num_lyr+1):
    params['W'][lay_num] = tf.Variable(tf.random_normal([net_shape[lay_num], net_shape[lay_num+1]],stddev = 0.05),name='weights_layer'+str(lay_num))
    params['b'][lay_num] = tf.Variable(tf.random_normal([net_shape[lay_num+1]],stddev = 0.05),name='biases_layer'+str(lay_num))

In [81]:
#network graph
def act_fn(a,lay_num):
    if act_fns[lay_num] == 'tanh':
        return tf.tanh(a)
    elif act_fns[lay_num] == 'relu':
        return tf.nn.relu(a)
    elif act_fns[lay_num] == 'elu':
        return tf.nn.elu(a)
    elif act_fns[lay_num] == 'sig':
        return tf.sigmoid(a)
    
def AutoEncoder(x,params):
    z = x
    for lay_num in range(num_lyr):
        z = act_fn(tf.add(tf.matmul(z,params['W'][lay_num]),params['b'][lay_num]),lay_num)
        if lay_num == code_lyr-1:
            wall_pusher = -tf.reduce_mean(tf.sqrt(tf.add(tf.square(z[:,0]),tf.square(z[:,1])))) #distance from (0,0)
            sparsity_penalty = wall_pusher
            code = z
    x_ = tf.add(tf.matmul(z,params['W'][lay_num+1]),params['b'][lay_num+1])
    cost = tf.sqrt(tf.reduce_mean(tf.squared_difference(x,x_))) + tf.mul(beta,sparsity_penalty)
    return (cost,code,x_)

In [82]:
cost,code,x_ = AutoEncoder(x,params)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [104]:
#parameter initialization
sess.run(tf.initialize_all_variables())

In [109]:
#learning
n_iter = 0
while True:
    try:
        _,cost_,code_,x__ = sess.run([optimizer,cost,code,x_],feed_dict={x:train_x,learning_rate:0.5,beta:0.1})
        if n_iter % 10 == 0:
            print('iter'+str(n_iter)+' cost:'+str(cost_))
        if n_iter+1 % 20 == 0:
            clear_output()
        n_iter += 1
    except KeyboardInterrupt:
        print "Training is stopped"
        break

iter0 cost:0.144642
iter10 cost:0.124795
iter20 cost:0.118225
iter30 cost:0.113718
iter40 cost:0.110312
iter50 cost:0.107681
iter60 cost:0.105626
iter70 cost:0.104005
iter80 cost:0.102711
iter90 cost:0.101662
iter100 cost:0.100797
iter110 cost:0.100073
iter120 cost:0.0994591
iter130 cost:0.09893
iter140 cost:0.0984682
iter150 cost:0.0980604
iter160 cost:0.0976982
iter170 cost:0.0973742
iter180 cost:0.0970825
iter190 cost:0.0968184
Training is stopped


In [110]:
#plots
plt.title('Sparse Vectors'); plt.xlabel('Dimension 1'); plt.ylabel('Dimension 2');
plt.scatter(code_[:n0][:,0],code_[:n0][:,1],c=[1,0,0], label = '0')
plt.scatter(code_[n0:e1][:,0],code_[n0:e1][:,1],c=[0,1,0], label = '1')
plt.scatter(code_[e1:e2][:,0],code_[e1:e2][:,1],c=[0,0,1], label = '2')
plt.scatter(code_[e2:][:,0],code_[e2:][:,1],c=[0,0,0], label = '3')
plt.legend()
plt.show()

In [76]:
np.set_printoptions(threshold=np.inf)
print (np.sum((code_[:n0,0] < 0.5)*(code_[:n0,1] < 0.5))+0.0)/n0
print (np.sum((code_[n0:e1,0] > 0.5)*(code_[n0:e1,1] > 0.5))+0.0)/n1
print (np.sum((code_[e1:e2,0] > 0.5)*(code_[e1:e2,1] < 0.5))+0.0)/n2
print (np.sum((code_[e2:,0] < 0.5)*(code_[e2:,1] > 0.5))+0.0)/n3

0.908890521675
0.927820035604
0.833820840951
0.779025466212


In [ ]:
img1 = train_x[1000].reshape(28,28)
plt.imshow(img1,cmap='gray')
plt.show()

In [65]:
# img2 = (x__[1100]*sigma + mu).reshape(28,28)
img2 = x__[1000].reshape(28,28)
plt.imshow(img2,cmap='gray')
plt.show()

In [ ]:
sess.close()

In [ ]:
code_.shape[0] - n0